In [ ]:
import matplotlib.pyplot as plt
import sqlite3
import csv

def run_query(sql_query, db_file):
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    cursor.execute(sql_query)

    rows = cursor.fetchall()

    conn.close()

    return rows


def create_table(db_name, tables):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    for table_name, schema in tables.items():
        cursor.execute(f"CREATE TABLE IF NOT EXISTS { table_name } ({ schema })")

    conn.commit()
    conn.close()


def populate_table_from_csv(db_name, table_name, csv_file):
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    with open(csv_file, 'r', newline='', encoding='utf-8') as csvfile:
        lines = list( csv.reader( csvfile ) )
        for values in lines[1:]:
            cursor.execute(f"INSERT INTO { table_name } VALUES ({','.join( ['?'] * len(values) )})", values)

    conn.commit()
    conn.close()


db_name = 'Youtube_Spotify_Top.db'

plt.rcParams.update({'font.family': 'Vazirmatn', 'font.size': 10})

In [ ]:
# DB Setup

Tables = {
    'YoutubeTop' : 'Rank INT PRIMARY KEY, Channel VARCHAR, Subs INT, Vid_Views INT, Vid_Count INT, Category VARCHAR, Start INT',

    'SpotifyTop' : 'Rank INT PRIMARY KEY, Artist VARCHAR, Listeners INT, PKListeners INT, Daily_Trend INT',

    'YoutubeArtists' : 'ID INT PRIMARY KEY, Artist VARCHAR, Url_Spotify VARCHAR, Track VARCHAR, Album VARCHAR, Album_Type VARCHAR, Duration_ms INT, Url_Youtube VARCHAR, Title VARCHAR, Channel VARCHAR, Views INT, Likes INT, Comments INT, Stream INT, Danceability FLOAT, Licensed BOOL, Official_Video BOOL'
}

create_table(db_name, Tables)

populate_table_from_csv(db_name, 'YoutubeTop', 'DataSets/YoutubeTop.csv')

populate_table_from_csv(db_name, 'SpotifyTop', 'DataSets/SpotifyTop.csv')

populate_table_from_csv(db_name, 'YoutubeArtists', 'DataSets/YoutubeArtists.csv')

In [ ]:
# 1. Top YT Channels Categories - Pie Chart

query = '''
Select Category, Count(*)
From YoutubeTop
Group By Category
Having Count(*) > 15 And Vid_Count > 0
'''

data = run_query(query, db_name)

categories, count = list( zip(*data) )

plt.figure(figsize=(8, 6))
plt.pie(count, labels=categories, autopct='%2.1f%%', startangle=90)
plt.title('Top YT Channels Categories')

plt.savefig('Plots/1. Top YT Channels Categories.png')

In [ ]:
# 2. Min and Max Danceability of Top Spotify Artists - Grouped Bar Chart

query = '''
Select SpotifyTop.Artist, Min(YoutubeArtists.Danceability), Max(YoutubeArtists.Danceability)
From SpotifyTop Join YoutubeArtists
On YoutubeArtists.Artist = SpotifyTop.Artist
Group By YoutubeArtists.Artist
Order By SpotifyTop.Rank
'''

data = run_query(query, db_name)[:10]

artists, min_danceability, max_danceability = list( zip(*data) )

bar_width = 0.35
index = range( len(artists) )

plt.figure(figsize=(12, 8))

plt.bar(index, max_danceability, bar_width)
plt.bar([i + bar_width for i in index], min_danceability, bar_width)

plt.title('Top Spotify Accounts from YT Top 1000')
plt.xlabel('Artists')
plt.ylabel('Danceability')
plt.xticks([i + bar_width / 2 for i in index], artists)

plt.savefig('Plots/2. Min and Max Danceability of Top Spotify Artists.png')

In [ ]:
# 3. YT Rank of Top Spotify Accounts - Bar Chart

query = '''
Select SpotifyTop.Artist, YoutubeTop.Rank
From YoutubeTop Join SpotifyTop
On SpotifyTop.Artist = YoutubeTop.Channel
Order By SpotifyTop.Rank
'''

data = run_query(query, db_name)[:10]

artists, ranks = list( zip(*data) )

plt.figure(figsize=(12, 8))
plt.bar(artists, ranks, bottom=0)

plt.title('YT Rank of Top Spotify Accounts')
plt.xlabel('Artists')
plt.ylabel('Ranks')

plt.savefig('Plots/3. YT Ranks of Top Spotify Accounts.png')

In [ ]:
# 4. Top Spotify Artists, Listeners VS PkListeners - Grouped Bar Chart

query = '''
Select SpotifyTop.Artist, SpotifyTop.Listeners, SpotifyTop.PKListeners
From SpotifyTop
'''

data = run_query(query, db_name)[:10]
  
artists, listeners, pklisteners = list( zip(*data) )
listeners, pklisteners = [ int(i.replace(',', '')) for i in listeners ], [ int(i.replace(',', '')) for i in pklisteners ]

bar_width = 0.35
index = range( len(artists) )

plt.figure(figsize=(12, 8))

plt.bar(index, listeners, bar_width, label='Listeners')
plt.bar([i + bar_width for i in index], pklisteners, bar_width, label='PkListeners')

plt.title('Listeners VS PkListeners of Top Spotify Artists')
plt.xlabel('Artists')
plt.ylabel('Listeners')
plt.xticks([i + bar_width / 2 for i in index], artists)
plt.legend()

plt.savefig('Plots/4. Top Spotify Artists, Listeners VS PkListeners.png')

In [ ]:
# 5. Categories of Spotify Accounts in Top YT Channels - Bar Chart

query = '''
Select DISTINCT YoutubeTop.Category, Count(*)
From YoutubeTop Join SpotifyTop
On SpotifyTop.Artist = YoutubeTop.Channel
Group By Category
'''

data = run_query(query, db_name)

categories, count = list( zip(*data) )

plt.figure(figsize=(8, 6))
plt.bar(categories, count)
plt.title('Categories of Spotify Accounts in Top YT Channels')

plt.savefig('Plots/5. Categories of Spotify Accounts in Top YT Channels.png')

In [ ]:
# 6. Top YT Channels Age vs Category - Scatter Plot

query = '''
Select YoutubeTop.Category, 2024 - YoutubeTop.Start
From YoutubeTop
Where YoutubeTop.Start >= 2005
And YoutubeTop.Category In ( Select YoutubeTop.Category From YoutubeTop Group By YoutubeTop.Category Having Vid_Count > 0 )
'''

data = run_query(query, db_name)

categories, ages = list( zip(*data) )

plt.figure(figsize=(12, 8))
plt.scatter(ages, categories)
plt.title('Top YT Channels - Age vs Category')

plt.savefig('Plots/6. Top YT Channels - Age vs Category.png')

In [ ]:
# 7. Max Subs and Listeners of Top Spotify Artists in Each YT Category - Bar Chart

query = '''
Select YoutubeTop.Category, SpotifyTop.Listeners, YoutubeTop.Subs, Min(SpotifyTop.Rank)
From YoutubeTop Join SpotifyTop
On SpotifyTop.Artist = YoutubeTop.Channel
Group By YoutubeTop.Category
'''

data = run_query(query, db_name)

categories, max_listeners, max_subs, ranks = list( zip(*data) )
max_listeners, max_subs = [ int(i.replace(',', '')) for i in max_listeners ], [ int(i.replace(',', '')) for i in max_subs ]

bar_width = 0.35
index = range( len(categories) )

plt.figure(figsize=(12, 8))

plt.bar(index, max_listeners, bar_width, label='Max Listeners')
plt.bar([i + bar_width for i in index], max_subs, bar_width, label='Max Subs')

plt.title('Max Subs and Listeners of Top Spotify Artists in Each YT Category')
plt.xlabel('Artists')
plt.ylabel('Subs & Listeners')
plt.xticks([i + bar_width / 2 for i in index], categories)
plt.legend()

plt.savefig('Plots/7. Max Subs and Listeners of Top Spotify Artists in Each YT Category.png')

In [ ]:
# 8. Official YT Music Videos [ Availabe in DB... ] of Top Spotify Artists Max Views - Stacked Bar Chart

album_query = '''
Select SpotifyTop.Artist, Max(YoutubeArtists.Views)
From YoutubeArtists Join SpotifyTop
On SpotifyTop.Artist = YoutubeArtists.Artist
Where SpotifyTop.Rank <= 10 And YoutubeArtists.Licensed = 'TRUE' And YoutubeArtists.Official_Video = 'TRUE' And YoutubeArtists.Album_Type = 'album'
Group By SpotifyTop.Artist
Order By SpotifyTop.Rank
'''

single_query = '''
Select SpotifyTop.Artist, Max(YoutubeArtists.Views)
From YoutubeArtists Join SpotifyTop
On SpotifyTop.Artist = YoutubeArtists.Artist
Where SpotifyTop.Rank <= 10 And YoutubeArtists.Licensed = 'TRUE' And YoutubeArtists.Official_Video = 'TRUE' And YoutubeArtists.Album_Type = 'single'
Group By SpotifyTop.Artist
Order By SpotifyTop.Rank
'''

album_data = run_query(album_query, db_name)
album_artists, album_views = list( zip(*album_data) )

single_data = run_query(single_query, db_name)
single_artists, single_views = list( zip(*single_data) )
single_views_bottom = [ album_views[i] for i in range( len(album_views) ) if album_artists[i] in single_artists ]

plt.figure(figsize=(12, 8))

plt.bar(album_artists, album_views, label='Album')
plt.bar(single_artists, single_views, bottom=single_views_bottom, label='Single')

plt.title('Official YT Music Videos of Top Spotify Artists Max Views')
plt.xlabel('Artists')
plt.ylabel('Views')
plt.legend()

plt.savefig('Plots/8. Official YT Music Videos of Top Spotify Artists Max Views.png')